In [1]:
import pandas as pd
import numpy as np
import requests
import json
import re
import unidecode
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# podcasts_df = pd.read_pickle("pickle_files/english_podcasts.pkl")

In [3]:
# podcasts_df['episode_titles'] = None
# podcasts_df['episode_descriptions'] = None

In [4]:
podcasts_df.head()

,title,producer,genre,description,num_episodes,rating,num_reviews,link,episode_titles,episode_descriptions
0,History Hyenas with Chris Distefano and Yannis...,RiotCast Network,Comedy,Comedians Yannis Pappas and Chris Distefano go...,98.0,4.9,1300.0,https://podcasts.apple.com/us/podcast/history-...,None,None
1,Curiosity Daily,Westwood One,Education,The award-winning Curiosity Daily podcast from...,300.0,4.8,428.0,https://podcasts.apple.com/us/podcast/curiosit...,None,None
2,Spirits,Multitude,History,"A boozy weekly podcast about mythology, legend...",161.0,4.8,1200.0,https://podcasts.apple.com/us/podcast/spirits/...,None,None
3,The Soundtrack Show,iHeartRadio,TV & Film,The Soundtrack Show is a weekly look at film s...,67.0,4.9,879.0,https://podcasts.apple.com/us/podcast/the-soun...,None,None
4,Writing Excuses,"Brandon Sanderson, Mary Robinette Kowal, Dan W...",Arts,"Brandon Sanderson, Mary Robinette Kowal, Howar...",58.0,4.7,775.0,https://podcasts.apple.com/us/podcast/writing-...,None,None


In [5]:
podcast_links = list(podcasts_df['link'])
failed_podcasts = list()

In [6]:
def clean_title(t):
    t = unidecode.unidecode(t)
    t = t.replace('\n',' ')
    t = re.sub(r'[^\w\s]','',t)
    t = re.sub(r'\d+','',t)
    t = t.lower()
    t = t.strip()
    return t

def clean_description(d):
    d = unidecode.unidecode(d)
    d = d.replace('\n',' ')
    d = re.sub(r'[^\w\s]','',d)
    d = re.sub(r'\d+','',d)
    if re.findall(r'(.*) brought to you by.*',d):
        d = re.sub(r'brought to you by.*','',d)
    if re.search(r'(.*) sponsored by.*',d):
        d = re.sub(r'sponsored by.*','',d)
    d = d.lower()
    d = d.strip()
    return d

In [7]:
def get_recent_podcast_episodes(link):
    episode_titles = ''
    episode_desc = ''
    
    driver = webdriver.PhantomJS()
    driver.get(link)
    html = driver.page_source.encode('utf-8')
    
    soup = BeautifulSoup(html, 'lxml')
    text = str(soup.find('script'))
    
    try:
        text = text.split('"workExample":')[1].split(',"aggregateRating"')[0]
        episode_data = json.loads(text)

        for episode in episode_data:
            title = episode['name']
            c_title = clean_title(title)

            description = episode['description']
            c_description = clean_description(description)

            episode_titles += (c_title + " ")
            episode_desc += (c_description + " ")
            
        episode_titles = episode_titles.strip()
        episode_desc = episode_desc.strip()
            
    except Exception:
        episode_title = np.nan
        episode_desc = np.nan
        print("Failed on: " + str(link))
        failed_podcasts.append(link)
        
    podcasts_df.loc[podcasts_df['link'] == link, 'episode_titles'] = episode_titles
    podcasts_df.loc[podcasts_df['link'] == link, 'episode_descriptions'] = episode_desc

In [8]:
for link in tqdm(podcast_links):
    get_recent_podcast_episodes(link)      

  0%|          | 0/4436 [00:00<?, ?it/s]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  0%|          | 1/4436 [00:03<4:30:11,  3.66s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  0%|          | 2/4436 [00:05<3:54:34,  3.17s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Sele

  1%|          | 23/4436 [00:51<2:41:21,  2.19s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  1%|          | 24/4436 [00:53<2:36:44,  2.13s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  1%|          | 25/4436 [00:55<2:34:59,  2.11s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnin

  1%|          | 46/4436 [01:52<4:12:50,  3.46s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  1%|          | 47/4436 [01:54<3:56:00,  3.23s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  1%|          | 48/4436 [01:59<4:28:29,  3.67s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnin

  2%|▏         | 69/4436 [03:06<3:35:50,  2.97s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  2%|▏         | 70/4436 [03:09<3:28:57,  2.87s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  2%|▏         | 71/4436 [03:12<3:35:02,  2.96s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnin

  2%|▏         | 92/4436 [04:23<4:29:27,  3.72s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  2%|▏         | 93/4436 [04:27<4:43:39,  3.92s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
  2%|▏         | 94/4436 [04:31<4:35:42,  3.81s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnin

...

Failed on: https://podcasts.apple.com/us/podcast/learn-spanish-spanishpod101-com/id270572701


/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
 99%|█████████▉| 4383/4436 [4:13:11<03:05,  3.50s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
 99%|█████████▉| 4384/4436 [4:13:14<02:46,  3.20s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has bee

 99%|█████████▉| 4405/4436 [4:14:23<01:52,  3.63s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
 99%|█████████▉| 4406/4436 [4:14:25<01:39,  3.32s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
 99%|█████████▉| 4407/4436 [4:14:29<01:38,  3.38s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  

Failed on: https://podcasts.apple.com/us/podcast/everyday-moments-with-joyce-meyer/id270646195


/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
100%|█████████▉| 4425/4436 [4:15:28<00:37,  3.39s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has been deprecated, please use headless '
100%|█████████▉| 4426/4436 [4:15:30<00:32,  3.22s/it]/Users/sidd/anaconda3/lib/python3.7/site-packages/selenium/webdriver/phantomjs/webdriver.py:49: UserWarning: Selenium support for PhantomJS has been deprecated, please use headless versions of Chrome or Firefox instead
  warnings.warn('Selenium support for PhantomJS has bee

In [12]:
print('Total Failed Podcasts: ' + str(len(failed_podcasts)))

Total Failed Podcasts: 133


In [10]:
# podcasts_df.to_pickle('pickle_files/english_podcasts_detailed.pkl')

In [13]:
podcasts_df = podcasts_df.dropna()
podcasts_df = podcasts_df.reset_index(drop=True)

In [16]:
# podcasts_df.to_pickle('pickle_files/english_podcasts_detailed_cleaned.pkl')